<a href="https://colab.research.google.com/github/joangog/object-detection/blob/main/mask_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Model evaluation (inference) on COCO 2017 dataset

The following models will be evaluated:

| Model | Backbone | Image Size | Parameters | GFLOPs
| --- | --- | --- | --- | --- |
| YOLOv5s |  Custom | 640x640 | 7.3M | 17 |
| YOLOv5m |  Custom | 640x640 | 21.4M | 51.3 |
| YOLOv5l |  Custom |640x640 | 47M | 115.5 |
| YOLOv3-tiny |  Darknet53 | 640x640 | 8.8M | 13.3 |

<br>

**Note: GPU Runtime needed**

*Example experiment: Tesla K80, 460.32.03, 11441 MiB, batch_size=8, workers=2*

In [ ]:
# Show system specs
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla K80, 460.32.03, 11441 MiB


### Get requirements
*Note: Restart runtime after installation*

In [1]:
%%shell

# Install Yolov5
cd /content
git clone https://github.com/ultralytics/yolov5
cd yolov5
pip install --quiet -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 9300, done.
remote: Total 9300 (delta 0), reused 0 (delta 0), pack-reused 9300
Receiving objects: 100% (9300/9300), 9.73 MiB | 17.48 MiB/s, done.
Resolving deltas: 100% (6464/6464), done.
     |████████████████████████████████| 3.0 MB 7.1 MB/s 
     |████████████████████████████████| 636 kB 70.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
%%shell

# Install Yolov3
cd /content
git clone https://github.com/ultralytics/yolov3
cd yolov3
pip install --quiet -r requirements.txt

Cloning into 'yolov3'...
remote: Enumerating objects: 9862, done.
remote: Total 9862 (delta 0), reused 0 (delta 0), pack-reused 9862
Receiving objects: 100% (9862/9862), 9.19 MiB | 14.68 MiB/s, done.
Resolving deltas: 100% (6666/6666), done.


In [4]:
%%shell

# Install flops-counter
pip install ptflops

  Created wheel for ptflops: filename=ptflops-0.6.6-py3-none-any.whl size=8903 sha256=f2c757b2b1657a12f9c339c79ba32e9b2a8a392f74dece54294f7401ad5a7c77
  Stored in directory: /root/.cache/pip/wheels/eb/7c/e5/2332373fcac1b39ba9eb95698ac370da3e14eaba5516e22721
Successfully built ptflops


In [3]:
%%shell

# Clone asset files
cd /content
git clone https://github.com/joangog/object-detection-assets
cd object-detection-assets
mv scripts ../
rm -rf /content/object-detection-assets/

Cloning into 'object-detection-assets'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 59 (delta 14), reused 51 (delta 9), pack-reused 0
Unpacking objects: 100% (59/59), done.


### Import packages

In [2]:
from google.colab import files
from google.colab import drive

import os, sys
import math
import time
import copy
import re

import numpy as np
import pandas as pd
import json
import PIL
import IPython

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch.utils.tensorboard import SummaryWriter
import torchvision
import torchvision.models.detection as M
import torchvision.transforms.functional as F
import torchvision.utils as U
from torchvision.datasets import CocoDetection

from pycocotools import coco
from pycocotools import mask as cocomask

from ptflops import get_model_complexity_info

import scripts.utils as SU
import scripts.transforms as ST
import scripts.engine as SE
import scripts.coco_utils as SCU
from scripts.coco_eval import CocoEvaluator

### Download Mask dataset

In [3]:
%%shell

cd /content
mkdir -p dataset
cd dataset
mkdir -p images annotations labels
cd labels
mkdir val_images


In [4]:
%%shell

cd /content/dataset/images

# Download validation images
gdown --id '101F2k6PJ-tD_uwlsCG7zzGF9ILJW01M1'
unzip -q -n 'val_images.zip'

Downloading...
From: https://drive.google.com/uc?id=101F2k6PJ-tD_uwlsCG7zzGF9ILJW01M1
To: /content/dataset/images/val_images.zip
14.9MB [00:01, 9.23MB/s]


In [5]:
%%shell

cd /content/dataset/annotations

# Download validation annotations
gdown --id '1YLV7-7vmiNdFI8Xpdx_jbhnxfgQRWrgF'
cp '/content/dataset/annotations/val.json' '/content/dataset/images/val_images'

Downloading...
From: https://drive.google.com/uc?id=1YLV7-7vmiNdFI8Xpdx_jbhnxfgQRWrgF
To: /content/dataset/annotations/val.json
100% 105k/105k [00:00<00:00, 923kB/s]


### Load Mask dataset

In [7]:
val_img_dir = '/content/dataset/images/val_images'
val_ann_file = 'val.json'  # annotations
val_ann_path = os.path.join(val_img_dir,val_ann_file)  

# Define data transforms
transforms = ST.Compose([ST.ToTensor()])

# Create dataset
val_dataset = CocoDetection(val_img_dir, val_ann_path, transforms = transforms)

# Create data loader
batch_size = 1
num_workers = 2
val_data_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=SU.collate_fn)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


### Load model checkpoint

In [17]:
# Mount GDrive to load model checkpoint
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
load_ckpt_path = '/content/drive/MyDrive/object-detection-checkpoints/pretrained/MASKD_yolov5s_sgd_ep100_lr01_run/weights/last.pt'


RuntimeError: ignored

### Load model

In [27]:
%cd /content

# Delete utils package to reload it (if loaded), because YOLOv3 and YOLOv5 have
# the same name for it and it causes error
try:
  sys.modules.pop('utils')
except:
  pass

# @markdown Model Selection { display-mode: 'form', run: 'auto' }
model_name = 'YOLOv5s' # @param ['SSD300 VGG16', 'SSDlite320 MobileNetV3-Large', 'Faster R-CNN ResNet-50 FPN', 'Faster R-CNN MobileNetV3-Large FPN', 'Mask R-CNN ResNet-50 FPN', 'YOLOv5s', 'YOLOv5m', 'YOLOv5l', 'YOLOv3', 'YOLOv3-tiny', 'YOLOv3-spp']

# @markdown *Note: If you get the error "Cache may be out of date, try 'force_reload=True'" then restart runtime.*

num_classes = len(val_dataset.coco.getCatIds())

if model_name == 'SSD300 VGG16':
  model_id = 'ssd300_vgg16'
  model = M.ssd300_vgg16(pretrained=False, progress=True)
  model_img_size = (3,300,300)
elif model_name == 'SSDlite320 MobileNetV3-Large':
  model_id = 'ssdlite320_mobilenet_v3_large'
  model = M.ssdlite320_mobilenet_v3_large(pretrained=False, progress=True)
  model_img_size = (3,320,320)
elif model_name == 'Faster R-CNN ResNet-50 FPN':
  model_id = 'fasterrcnn_resnet50_fpn'
  model = M.fasterrcnn_resnet50_fpn(pretrained=False, progress=True)
  model_img_size = (3,800,800) # COCO's 640x640 in upscaled to the model's minimum 800x800
elif model_name == 'Faster R-CNN MobileNetV3-Large FPN':
  model_id = 'fasterrcnn_mobilenet_v3_large_fpn'
  model = M.fasterrcnn_mobilenet_v3_large_fpn(pretrained=False, progress=True)
  model_img_size = (3,800,800) 
elif model_name == 'Mask R-CNN ResNet-50 FPN':
  model_id = 'maskrcnn_resnet50_fpn'
  model = M.maskrcnn_resnet50_fpn(pretrained=False, progress=True)
  model_img_size = (3,800,800)
elif 'YOLOv5' in model_name:
  model_id = model_name.lower().replace('-','_')
  model = torch.hub.load('/content/yolov5', 'custom', path=load_ckpt_path, source='local', force_reload=True)
  model_img_size = (3,640,640)
elif 'YOLOv3' in model_name:
  model_id = model_name.lower().replace('-','_')
  model = torch.hub.load('/content/yolov3', 'custom', path=load_ckpt_path, source='local', force_reload=True)
  model_img_size = (3,640,640)

# Prepare model for dataset (for Fast R-CNN or Mask R-CNN)
if 'R-CNN' in model_name: 
  # Get the number of input features for the bbox predictor
  in_features = model.roi_heads.box_predictor.cls_score.in_features
  # Replace the pre-trained head with a new one
  model.roi_heads.box_predictor = M.faster_rcnn.FastRCNNPredictor(in_features, num_classes+1)  # includes background (0) class
  if 'Mask R-CNN' in model_name:
    # Get the number of input features for the segmentation max predictor
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # Replace the mask predictor with a new one
    model.roi_heads.mask_predictor = M.mask_rcnn.MaskRCNNPredictor(in_features_mask, hidden_layer,num_classes+1)

print('-------------------------------------------------------------------------------------------------------\n')

print(f'Loaded model: {model_name}')
model_params = round(sum([param.numel() for param in model.parameters()]) / 1000000, 1)
print(f'\t- Parameters: {model_params}M')
model_macs, _ = get_model_complexity_info(model, model_img_size, as_strings=False, 
                                          print_per_layer_stat=False, verbose=False)
model_gflops = round(2 * int(model_macs) / 1000000000, 1)
print(f'\t- GFLOPs: {model_gflops}')

YOLOv5 🚀 2021-9-13 torch 1.9.0+cu102 CPU

Fusing layers... 


/content


Model Summary: 224 layers, 7059304 parameters, 0 gradients, 16.3 GFLOPs
Adding AutoShape... 


-------------------------------------------------------------------------------------------------------

Loaded model: YOLOv5s
	- Parameters: 7.1M
	- GFLOPs: 16.3
